In [1]:
import pandas as pd
import geopandas as gpd
import os

In [2]:
gdf_e = gpd.read_file("estacoes_pluv_bh_ana.geojson")

In [3]:
len(gdf_e)

194

In [4]:
len(gdf_e.Codigo.unique())

79

In [5]:
#gdf_e[gdf_e.Operando == "Não"]
#len(gdf_e[gdf_e.TipoEstacao == "Pluviométrica"].Codigo.unique())

In [ ]:
## verificando se baixei todos os dados pluviométricos

import os
pasta = './dados_pluviometricos/estacoes_pluv_ana'
arquivos = [arquivo for arquivo in os.listdir(pasta) if arquivo.endswith(".zip")]

codigos = []
for arquivo in arquivos:
    codigos.append(int(arquivo.split("_")[1]))

all_codigos = [arquivo for arquivo in set(list(gdf_e.Codigo))]

[codigo for codigo in all_codigos if codigo not in codigos]

In [ ]:
print(len(arquivos), len(gdf_e))

In [ ]:
import zipfile

unzip_dir = './dados_pluviometricos/estacoes_pluv_ana/unzip'
if not os.path.exists(unzip_dir):
    os.makedirs(unzip_dir)

for arquivo in arquivos:
        zip_file_path = './dados_pluviometricos/estacoes_pluv_ana/' + arquivo
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_dir)


In [ ]:
int(arquivo.split("_")[1])

In [ ]:
csv_files = []
for codigo in all_codigos:
    arquivos_no_roi = [arquivo for arquivo in arquivos if int(arquivo.split("_")[1]) == codigo]
    if arquivos_no_roi:
        csv_file = unzip_dir + "/" + str(codigo) + "_Chuvas.csv"
        csv_files.append(csv_file)
  #  print(codigo, csv_file)
   # break 


In [ ]:
csv_files

In [ ]:
df_es = pd.DataFrame()
missing_files = []
for csv_file in csv_files:
    try:
        df_e = pd.read_csv(
                csv_file,
                encoding='ISO-8859-1',
                sep=';',           # Delimitador de campo, ponto e vírgula
            #    quotechar='"',     # Definir o caractere de aspas para valores com vírgulas internas
                skiprows=14,       # Pular as primeiras 10 linhas (ajustar conforme necessário)
          #     header=15,          # Usar a linha 7 como cabeçalho (ajustar conforme necessário)
       #         on_bad_lines='skip'  # Ignorar linhas malformadas
            )
       
        df_es = pd.concat([df_es, df_e[['EstacaoCodigo','Data', 'Maxima', 'Total', 'DiaMaxima', 'NumDiasDeChuva', 
                                        'Chuva01', 'Chuva02', 'Chuva03', 'Chuva04', 'Chuva05', 'Chuva06',
                                           'Chuva07', 'Chuva08', 'Chuva09', 'Chuva10', 'Chuva11', 'Chuva12',
                                           'Chuva13', 'Chuva14', 'Chuva15', 'Chuva16', 'Chuva17', 'Chuva18',
                                           'Chuva19', 'Chuva20', 'Chuva21', 'Chuva22', 'Chuva23', 'Chuva24',
                                           'Chuva25', 'Chuva26', 'Chuva27', 'Chuva28', 'Chuva29', 'Chuva30',
                                           'Chuva31']]])
    except Exception as e:
        print(e, csv_file)
        missing_files.append(csv_file)
   # print(df_e.columns)


In [ ]:
len(df_es.EstacaoCodigo.unique())

In [ ]:
# O número de eventos mensais de chuva foi obtido a partir das séries históricas diárias de precipitação das estações pluviométrica localizadas 
# na área de estudo (Figura1). Foram considerados eventos significativos, as precipitações acima de 0,1mm (SHARP et al., 2020). Assim, para cada 
# mês e cada estação foi feita a soma das ocorrências de eventos significativos, totalizando os valores mensais.

import numpy as np

colunas_chuva = ['Chuva01', 'Chuva02', 'Chuva03', 'Chuva04', 'Chuva05', 
                 'Chuva06', 'Chuva07', 'Chuva08', 'Chuva09', 'Chuva10', 
                 'Chuva11', 'Chuva12', 'Chuva13', 'Chuva14', 'Chuva15', 
                 'Chuva16', 'Chuva17', 'Chuva18', 'Chuva19', 'Chuva20', 
                 'Chuva21', 'Chuva22', 'Chuva23', 'Chuva24', 'Chuva25', 
                 'Chuva26', 'Chuva27', 'Chuva28', 'Chuva29', 'Chuva30', 
                 'Chuva31']
df_es_fmt = df_es.copy()
df_es_fmt[colunas_chuva] = df_es_fmt[colunas_chuva].replace({',': '.'}, regex=True).apply(pd.to_numeric, errors='coerce')
for coluna in colunas_chuva:
    print(f"Valores únicos em {coluna}:")
    print(np.sort(df_es_fmt[coluna].unique())[0:3])
    print()

In [ ]:
df_es_fmt['NumDiasDeChuva_0_1'] = df_es_fmt[colunas_chuva].gt(0.1).sum(axis=1)
df_es_fmt[['EstacaoCodigo','Data', 'Maxima', 'Total', 'DiaMaxima', 'NumDiasDeChuva', 'NumDiasDeChuva_0_1']].sample(10)

In [ ]:
#gdf_e["Codigo"] = gdf_e['Codigo'].apply(lambda row: row.replace(".", "")).astype(int)
gdf_e.sample(3)

In [ ]:
df_es_dados = df_es_fmt[
                        ['EstacaoCodigo','Data', 'Maxima', 'Total', 'DiaMaxima', 'NumDiasDeChuva', 'NumDiasDeChuva_0_1']
                        ].merge(gdf_e[["Codigo", "Nome", "geometry"]], 
                                left_on="EstacaoCodigo", right_on="Codigo", how="left").rename(columns = {'NumDiasDeChuva': 'NumDiasDeChuva_ANA'})
                                                                                
df_es_dados.sample(3)

In [ ]:
gdf_es = gpd.GeoDataFrame(df_es_dados, geometry = "geometry", crs = "EPSG:4674")
gdf_es.plot()

In [ ]:
gdf_es.to_file("./dados_pluviometricos/estacoes_pluv_ana_com_dados.geojson")

In [ ]:
df_es_fmt.to_excel("./dados_pluviometricos/estacoes_pluv_ana_com_dados.xlsx", index = False)

In [ ]:
## PARA OS ANOS DE INTERESSE

In [ ]:
years = [str(y) for y in range(1985, 2025)]

In [ ]:
df_es_fmt = df_es_fmt[df_es_fmt["Data"].notna()].reset_index(drop=True)

In [ ]:
type(df_es_fmt.loc[0, "Data"])

In [ ]:
df_es_fmt = pd.read_excel("./dados_pluviometricos/estacoes_pluv_ana_com_dados.xlsx")
for year in years: 
    print(year)
    os.makedirs(f"./dados_pluviometricos/{year}/", exist_ok=True)
    mask = df_es_fmt.Data.notna() & df_es_fmt.Data.str.contains(year)
    df_es_fmt[mask].to_csv(f"./dados_pluviometricos/{year}/dados_pluv_ana_{year}.csv", index = False)

In [ ]:
for year in years: 
    df_es_fmt_y = pd.read_csv(f"./dados_pluviometricos/{year}/dados_pluv_ana_{year}.csv")
    print(year, len(df_es_fmt_y.EstacaoCodigo.unique()))